In [363]:
import pandas as pd
import numpy as np

In [364]:
#cargamos los 3 datasets que vamos a utlizzar para crear las funciones
dfjuegos=pd.read_csv('Juegos_steam.csv')
dfreviews=pd.read_csv('revies_por_usuario.csv')
dfitems=pd.read_csv('items_por_usuario.csv')

In [365]:
#def PlayTimeGenre( genero : str ): #Debe devolver año con mas horas jugadas para dicho género.
#Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [366]:
dfjuegos['Género']

0        ['Action', 'Casual', 'Indie', 'Simulation', 'S...
1             ['Free to Play', 'Indie', 'RPG', 'Strategy']
2        ['Casual', 'Free to Play', 'Indie', 'Simulatio...
3                        ['Action', 'Adventure', 'Casual']
4                          Action, Indie, Casual, Sports, 
                               ...                        
32127        ['Casual', 'Indie', 'Simulation', 'Strategy']
32128                      ['Casual', 'Indie', 'Strategy']
32129                    ['Indie', 'Racing', 'Simulation']
32130                                  ['Casual', 'Indie']
32131    Early Access, Adventure, Indie, Action, Simula...
Name: Género, Length: 32132, dtype: object

In [367]:
#por ahora trabajmos con los años que si tenemos datos, despues se vera el tratamiento de los años restantes
dfjuegos=dfjuegos.dropna(subset='Año_lanzamiento')

In [368]:
#hacemos una lista con los generos para poder acceder a ellos
generos=[]
gen=dfjuegos['Género'].drop_duplicates().str.strip("']['").str.split(", ",expand=True).copy()
for a in range(len(gen.columns)):
    for b in gen[a].unique():
        if b !=None:
            if b.strip("' ") not in generos:
                if b.strip("' ")!='':
                    generos.append(b.strip("' "))
              

In [369]:
# ordenamos por alfabeto
generos=sorted(generos)
generos

['Accounting',
 'Action',
 'Adventure',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'N/D',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing']

In [370]:
# primero sumamos las horas de cada juego en total
itemsf1=dfitems[['Id_juego','Horas_jugadas']]
dff1=itemsf1.groupby('Id_juego').sum()
dff1.head()

,Horas_jugadas
Id_juego,
10,17107858
20,960524
30,756375
40,154424
50,726545


In [371]:
#añadimos las columnas de todos los generos, un genero por columna, se llena con 0 por default
for a in generos:
    dff1[a]=np.zeros(len(dff1),dtype=int)
#se agrega la columna Género al dataframe

dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,960524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,756375,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,154424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,726545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [372]:
#agregamos la columna generos, y rellenamos con 'N/D' por default, en caso que haya ids que no esten en nuestro dataset de juegos
Generos=['N/D' for a in range(len(dff1))]
dff1['Género']=Generos
dff1.Género

Id_juego
10        N/D
20        N/D
30        N/D
40        N/D
50        N/D
         ... 
528580    N/D
528660    N/D
529670    N/D
529820    N/D
530720    N/D
Name: Género, Length: 10978, dtype: object

In [373]:
#rellenamos la columna cGénero con los generos del dataframe dfjuegos
for b,a in enumerate(dff1.index):
    if dfjuegos.Id_juego[dfjuegos.Id_juego==a].array.size>0:
        dff1.loc[a,'Género']=dfjuegos.Género[dfjuegos.Id_juego==a].iloc[0]
dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Género
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
20,960524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
30,756375,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
40,154424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
50,726545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']


In [374]:
dff1.loc[dff1.Género.str.contains('Action'),'Action']

Id_juego
10        0
20        0
30        0
40        0
50        0
         ..
526460    0
527340    0
527520    0
527810    0
528660    0
Name: Action, Length: 3878, dtype: int32

In [375]:
#ahora rellenamos las columnas de cada genero, donde el genero si exista en la colomna Género
for a in generos:
    dff1.loc[dff1.Género.str.contains(a),a]=1
dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Género
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
20,960524,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
30,756375,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
40,154424,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']
50,726545,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,['Action']


In [376]:
#agregamos la columna Año, y rellenamos con 'N/D' por default, en caso que haya ids que no esten en nuestro dataset de juegos
años=['N/D' for a in range(len(dff1))]
dff1['Año']=años
dff1.Año

Id_juego
10        N/D
20        N/D
30        N/D
40        N/D
50        N/D
         ... 
528580    N/D
528660    N/D
529670    N/D
529820    N/D
530720    N/D
Name: Año, Length: 10978, dtype: object

In [377]:
# Agregamos la columna año al dataframe de la funcion
for b,a in enumerate(dff1.index):
    if dfjuegos.Id_juego[dfjuegos.Id_juego==a].array.size>0:
        dff1.loc[a,'Año']=dfjuegos.Año_lanzamiento[dfjuegos.Id_juego==a].iloc[0]
dff1.head()

,Horas_jugadas,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Género,Año
Id_juego,,,,,,,,,,,,,,,,,,,,,
10,17107858,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],2000
20,960524,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],1999
30,756375,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],2003
40,154424,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],2001
50,726545,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,['Action'],1999


In [378]:
dff1.columns

Index(['Horas_jugadas', 'Accounting', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'N/D',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing',
       'Género', 'Año'],
      dtype='object')

In [379]:
#reordenamos las columnas
dff1=dff1[['Género', 'Año','Horas_jugadas', 'Accounting', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'N/D',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']]
# confirmamos orden
dff1.columns

Index(['Género', 'Año', 'Horas_jugadas', 'Accounting', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'N/D',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production',
       'Web Publishing'],
      dtype='object')

In [380]:
dff1_final=pd.DataFrame()

In [387]:
for a in generos:
    if dff1[dff1[a]==1].groupby(by=['Año']).sum()['Horas_jugadas'].array.size==0:
        continue
    else:
        dff1_final[a]=dff1[dff1[a]==1].groupby(by=['Año']).sum()['Horas_jugadas'].copy()
dff1_final

,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,Indie,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
Año,,,,,,,,,,,,,,,,,,,,,
1983,3473,3473.0,NaN,NaN,3473.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,384,384.0,NaN,NaN,384.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988,16001,29963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13962.0,...,NaN,13962.0,NaN,1.0,NaN,NaN,13962.0,NaN,NaN,NaN
1989,607,9311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5084.0,NaN,NaN,NaN,NaN,NaN,NaN
1990,18335,5492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1926.0,NaN,NaN,60.0,NaN,NaN,NaN
1991,2502,1722.0,NaN,NaN,1355.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,144.0,NaN,NaN,0.0,NaN,NaN,NaN
1992,1925,23795.0,NaN,NaN,2492.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2164.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
1993,24623,46772.0,NaN,NaN,22039.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,65.0,NaN,NaN,182863.0,NaN,NaN,NaN
1994,120177,15890.0,NaN,NaN,45975.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2092.0,289.0,9351.0,NaN,NaN,1169.0,NaN,NaN,NaN


In [388]:
#   pasamos el año del indice a una columna
dff1_final.reset_index(inplace=True)


In [391]:
#hacemos un dataframe vacio donde guardar las consultas finales
dff1=pd.DataFrame()

In [450]:
generos_funcion=[]
años_funcion=[]
for a in dff1_final.drop(columns='Año').columns:
    generos_funcion.append(a)
    años_funcion.append(dff1_final[['Año',a]].sort_values(ascending=False,by=a).iloc[0,0])
print(generos_funcion)
print(años_funcion)

['Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'N/D', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']
['2012', '2011', '2015', '2014', '2015', '2012', '2013', '2013', '2013', '2006', '2013', '2011', '2014', '2011', '2015', '2006', '2014', '2015', '2013', '2014', '2015', '2012']


In [455]:
dff1=pd.DataFrame({'Género':generos_funcion,'Año':años_funcion,})
dff1


,Género,Año
0,Action,2012
1,Adventure,2011
2,Animation &amp; Modeling,2015
3,Audio Production,2014
4,Casual,2015
5,Design &amp; Illustration,2012
6,Early Access,2013
7,Education,2013
8,Free to Play,2013
9,Indie,2006


In [456]:
# guardamos en un csv el datafrme listo para la funcion
dff1.to_csv('Función_PlayTimeGenre.csv',index=False)

In [ ]:

def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
#Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
def UsersRecommend( año : int ): #Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [ ]:
def UsersWorstDeveloper( año : int ): #Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [ ]:
def sentiment_analysis( empresa desarrolladora : str ): #Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
#Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}